# Подключение библиотек

In [6]:
import numpy as np
import itertools
import random
import sys

# Задание параметров

In [7]:
a = 3;#ширина прохода
b = 1.5;#расстояние между полками
k = 1000;#количество ячеек на стеллаже
n = 20;#количество стеллажей
l = 1;#ширина стеллажа
x = 0.3;#ширина ячейки
N = 5;#количество товаров в заказе

# Список заказов

In [8]:
vertex_ryadi=np.random.randint(1,2*n,N)
vertex_ryadi=np.append(vertex_ryadi[::-1],1)[::-1]
vertex_yacheyki=np.random.randint(1,2*k,N)
vertex_yacheyki=np.append(vertex_yacheyki[::-1],k+1)[::-1]
v=np.column_stack((vertex_ryadi,vertex_yacheyki))
print(v)

[[   1 1001]
 [   6 1167]
 [  20 1500]
 [  21  986]
 [  16 1702]
 [  21 1677]]


# Создание матрицы

In [9]:
M=np.zeros((N+1,N+1))
for i in range(N+1):
    for j in range(i,N+1):
        s=0;
        if (i==j):
            M[i][j]=0;
        else:
            # если мы на входе, то отдельно рассчитываем
            if i==1:
                s=s+b;
                if(v[j][1]>k):
                    s=s+a/2;
                else:
                    s=s-a/2;
            if j==1:
                s=s+b;
                if(v[i][1]>k):
                    s=s+a/2;
                else:
                    s=s-a/2;
            # Рассматриваем различные случаи: сверху или снизу, четный или нечётный ряд
            if (v[i][0] % 2 == v[j][0] % 2):
                s=s+abs(v[i][0]-v[j][0])/2*(l+b);
            else:
                s=s+b+abs(v[i][0]-v[j][0]-1)/2*(l+b);    
            if (v[i][1]>k) and (v[j][1]>k):
                s=s+min(v[i][1]-k+v[j][1]-k, 2*k-v[i][1]+2*k-v[j][1])*x
            else:
                if (v[i][1]<k) and (v[j][1]<k):
                    s=s+min(v[i][1]+v[j][1], k-v[i][1]+k-v[j][1])*x
                else:
                    s=s+abs(v[i][1]-v[j][1])*x;
            M[i][j]=s
            M[j][i]=s

In [10]:
print(M)

[[  0.   62.4 176.8  29.5 232.4 228.4]
 [ 62.4   0.  220.6  75.8 276.2 277.7]
 [176.8 220.6   0.  158.2 244.4 250.9]
 [ 29.5  75.8 158.2   0.  221.3 207.3]
 [232.4 276.2 244.4 221.3   0.  195.3]
 [228.4 277.7 250.9 207.3 195.3   0. ]]


# Алгоритм Хелда-Карпа

In [11]:
def held_karp(dists):
    n = len(dists)
    # Мэпим каждый набор вершин с путём, который нужно пройти до этого подмножества, а также с предыдущей вершиной. 
    # Наборы вершин представлены битами
    C = {}
    # Задаём пути до каждой вершины из первой вершины
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)
    # Проходим по всем длинам и наборам и запоминаем промежуточные результаты
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Задаём биты для всех вершин в наборе
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            # Ищем наименьший путь, чтобы добраться до набора
            for k in subset:
                prev = bits & ~(1 << k)
                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                C[(bits, k)] = min(res)
    bits = (2**n - 1) - 1
    # Ищем наименьший путь
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)
    # Движемся в обратном направлении, чтобы восстановить весь путь
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits
    # Добавляем начальную вершину
    path.append(0)
    path=list(reversed(path))
    path.append(0)
    return opt, path

In [12]:
dists=M
dist, zakazi = held_karp(dists)

In [13]:
print(zakazi)

[0, 1, 2, 4, 5, 3, 0]


In [14]:
print(dist)

959.4999999999999


# Проверка работы алгоритма

In [15]:
s=0;
for i in range(0,len(zakazi)-1):
    s=s+M[zakazi[i]][zakazi[i+1]]
print(s) 

959.4999999999999


In [17]:
print(M)

[[  0.   62.4 176.8  29.5 232.4 228.4]
 [ 62.4   0.  220.6  75.8 276.2 277.7]
 [176.8 220.6   0.  158.2 244.4 250.9]
 [ 29.5  75.8 158.2   0.  221.3 207.3]
 [232.4 276.2 244.4 221.3   0.  195.3]
 [228.4 277.7 250.9 207.3 195.3   0. ]]


# Подготовка данных к полному перебору

In [18]:
print(len(M))
Vert=[]
for i in range(1,N+1):
    Vert.append(i)
print(Vert)
temp=list(itertools.permutations(Vert))
routes=[]
for i in temp:
    routes.append(list(i))
for i in routes:
    i.append(0)
    i.insert(0,0)

6
[1, 2, 3, 4, 5]


# Полный перебор

In [19]:
min_cost=0
min_route=routes[0]
for j in range(0,len(routes[0])-1):
    min_cost+=M[routes[0][j]][routes[0][j+1]]
for i in routes:
    cost=0;
    for j in range(0,len(i)-1):
        cost+=M[i[j]][i[j+1]]
    if(cost<min_cost):
        min_cost=cost;
        min_route=i;
print(min_cost, min_route)

959.4999999999999 [0, 1, 2, 4, 5, 3, 0]
